# Echo-A 1B5 (Memory model from scratch)
This attempts to build the memory model in stages, from scratch
(Instead of previous attempts in doing foundation + gpt4all + etc)

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

## Prepare the dataset

Prepare and preload the finetuning process dataset

In [18]:
%%script bash
# Reset the dataset dir
mkdir -p ./dataset
rm -rf ./dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

# We do a strong bias for smaller word count, to teach the concept from scratch
# so that the model can learn the function
python ./memory_script/gen_dataset_file.py ./dataset/word-2-count.jsonl  2  20000 &
python ./memory_script/gen_dataset_file.py ./dataset/word-5-count.jsonl  5  20000 &
python ./memory_script/gen_dataset_file.py ./dataset/word-10-count.jsonl 10 20000 &
python ./memory_script/gen_dataset_file.py ./dataset/word-20-count.jsonl 20 20000 &
python ./memory_script/gen_dataset_file.py ./dataset/word-40-count.jsonl 40 20000 &
python ./memory_script/gen_dataset_file.py ./dataset/word-80-count.jsonl 80 20000 &

# With a slight mix of the larger word count
python ./memory_script/gen_dataset_file.py ./dataset/word-100-count.jsonl 100 8000 &
python ./memory_script/gen_dataset_file.py ./dataset/word-200-count.jsonl 200 5500 &
python ./memory_script/gen_dataset_file.py ./dataset/word-300-count.jsonl 300 5000 &
python ./memory_script/gen_dataset_file.py ./dataset/word-400-count.jsonl 400 4500 &
python ./memory_script/gen_dataset_file.py ./dataset/word-500-count.jsonl 500 4000 &
python ./memory_script/gen_dataset_file.py ./dataset/word-600-count.jsonl 600 3500 &
python ./memory_script/gen_dataset_file.py ./dataset/word-700-count.jsonl 700 3000 &
python ./memory_script/gen_dataset_file.py ./dataset/word-800-count.jsonl 800 2500 &
python ./memory_script/gen_dataset_file.py ./dataset/word-900-count.jsonl 900 2000 &

wait
echo "## Done ##"

## Generating word reptition dataset ##
Generated JSONL file with - 2 max words, 20000 samples - at ./dataset/word-2-count.jsonl


Generated JSONL file with - 5 max words, 20000 samples - at ./dataset/word-5-count.jsonl
Generated JSONL file with - 10 max words, 20000 samples - at ./dataset/word-10-count.jsonl
Generated JSONL file with - 20 max words, 20000 samples - at ./dataset/word-20-count.jsonl
Generated JSONL file with - 100 max words, 8000 samples - at ./dataset/word-100-count.jsonl
Generated JSONL file with - 200 max words, 5500 samples - at ./dataset/word-200-count.jsonl
Generated JSONL file with - 40 max words, 20000 samples - at ./dataset/word-40-count.jsonl
Generated JSONL file with - 300 max words, 5000 samples - at ./dataset/word-300-count.jsonl
Generated JSONL file with - 900 max words, 2000 samples - at ./dataset/word-900-count.jsonl
Generated JSONL file with - 80 max words, 20000 samples - at ./dataset/word-80-count.jsonl
Generated JSONL file with - 400 max words, 4500 samples - at ./dataset/word-400-count.jsonl
Generated JSONL file with - 800 max words, 2500 samples - at ./dataset/word-800-count.j

In [19]:
DEEPSPEED_STRAT="deepspeed_stage_2_offload"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="Echo-A-1B5"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
TRAINER_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../RWKV-v4neo/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)

DEEPSPEED_STRAT: deepspeed_stage_2_offload
ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/ubuntu/picocreator-memory-experiment/notebook/experiment/memory
TRAINER_DIR: /home/ubuntu/picocreator-memory-experiment/RWKV-v4neo


## Stage 1: Low word count memory training

In [21]:
# Lets preload the requried dataset (<=80 words focused + <= some 900 words mixed in)
!cd "{TRAINER_DIR}" && \
    python3 preload_dataset.py "{NOTEBOOK_DIR}/Echo-A-1B5-mem-scratch.yaml"

# Ensure the checkpoint directory exists
!cd "{TRAINER_DIR}" && mkdir -p "../checkpoint/Echo-A-1B5-mem-scratch/"

Extracting data files: 100%|█████████████████████| 1/1 [00:00<00:00, 266.34it/s]
Dataset json downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/json/default-59c16bb75dfb623f/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.
Map (num_proc=32):  55%|████▍   | 87303/158000 [00:36<00:39, 1808.14 examples/s]

In [16]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    python new_train.py fit \
        -c "{NOTEBOOK_DIR}/Echo-A-1B5-mem-finetune.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Finetune-1 (bs=128, train-ctx=2048)" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --model.ctx_len=2048

[2023-07-10 01:54:13,137] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1'
/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 1148776073
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 1148776073
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.5
wandb: Run data is saved locally in ./wandb/run-20230710_015415-jf9erm5p
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run Echo-A-1B5 - Mem-Finetune-1 (bs=128, train-ctx=2048)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-Memory-Experiment
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-Memory-Experiment/runs/jf9erm5p
Using /home/picocreator/.cache/torch_extensions/py311_cu117 as PyTorc

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    && python export_checkpoint.py "../checkpoint/Echo-A-1B5-mem-finetune/last.ckpt"

In [ ]:
# Lets move and save this model
!cd "{TRAINER_DIR}" && cp ./checkpoint/Echo-A-1B5-mem-finetune/last.ckpt/rwkv_model.pth ./model/Echo-A-1B5-Stage3.pth
!cd "{TRAINER_DIR}" && ls -alh ./model/Echo-A-1B5-Stage3.pth

In [ ]:
# Lets do a quick dragon prompt validation
!cd "{TRAINER_DIR}" && python3 dragon_test.py ../model/Echo-A-1B5-Stage3.pth "cuda fp32"